<img src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 20px"/>

# Structured Streaming

## Prerequisites
* Web browser: **Chrome**
* A cluster configured with **8 cores** and **DBR 6.2**
* Run the code on the databricks Community edition workspace.


## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Setup

For each lesson to execute correctly, please make sure to run the **`Classroom-Setup`** cell at the<br/>
start of the cell (see the next cell) and the **`Classroom-Cleanup`** cell at the end.

In [ ]:
%run "../Includes/Classroom-Setup"

Preparing the Python environment...

Defining courseware-specific utility methods...

Defining user-facing utility methods...

Defining custom variables for this lesson...

Initializing Databricks Academy's testing framework...

Initializing Databricks Academy's services for generating dynamic data...

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Cleaning up the learning environment...no actions taken.

WARNING: This notebook was tested on DBR 6.2, but we found DBR 8.0. 
 Using an untested DBR may yield unexpected results and/or various errors 
 Please update your cluster configuration and/or download a newer version of this course before proceeding.

All done!

Define the name of the stream we are to use later in this lesson:

Start by reading the stream. 

For this step you will need to:
1. Starting with `spark`, an instance of `SparkSession`, and get the `DataStreamReader`
2. We will make sure to only consume only 1 file per trigger for this Data.
3. We have Specified the stream's schema using the instance `dataSchema`
4. Use `dsr.parquet()` to specify the stream's file type and source directory, `dataPath` 

When you are done, run the TEST cell that follows to verify your results.

In [ ]:
# TODO
dataSchema = "DepartureAt timestamp, FlightDate string, DepTime string, CRSDepTime string, ArrTime string, CRSArrTime string, UniqueCarrier string, FlightNum integer, TailNum string, ActualElapsedTime string, CRSElapsedTime string, AirTime string, ArrDelay string, DepDelay string, Origin string, Dest string, Distance string, TaxiIn string, TaxiOut string, Cancelled integer, CancellationCode string, Diverted integer, CarrierDelay string, WeatherDelay string, NASDelay string, SecurityDelay string, LateAircraftDelay string"

dataPath = "dbfs:/mnt/training/asa/flights/2007-01-stream.parquet"

initialDF = (spark.readStream
             .option("maxFilesPerTrigger", 1)
             .schema(dataSchema)
             .parquet(dataPath)
)


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Calculate the total of all delays</h2>

We want to calculate (and later graph) the total delay of each flight
1. Start with `initialDF` from the previous cell. 
2. Convert the following columns from `String` to `Integer`: `CarrierDelay`, `WeatherDelay`, `NASDelay`, `SecurityDelay` and `LateAircraftDelay`
3. Add the column `TotalDelay` which is the sum of the other 5 delays
4. Filter the flights by `UniqueCarrier` down to the carriers **AS**, **AQ**, **HA** and **F9**
5. Filter the results to non-zero delay's (`TotalDelay` > 0)
6. Assign the final DataFrame to `delaysDF`



In [ ]:
# TODO
from pyspark.sql.functions import *
delaysDF = (initialDF
  .withColumn("CarrierDelay", col("CarrierDelay").cast("integer"))  # Convert CarrierDelay to an Integer
  .withColumn("WeatherDelay", col("WeatherDelay").cast("integer"))  # Convert WeatherDelay to an Integer
  .withColumn("NASDelay", col("NASDelay").cast("integer"))  # Convert NASDelay to an Integer
  .withColumn("SecurityDelay", col("SecurityDelay").cast("integer"))  # Convert SecurityDelay to an Integer
  .withColumn("LateAircraftDelay", col("LateAircraftDelay").cast("integer"))  # Convert LateAircraftDelay to an Integer
  .withColumn("TotalDelay", col("CarrierDelay")+col("WeatherDelay")+col("NASDelay")+col("SecurityDelay")+col("LateAircraftDelay"))  # Sum all five as TotalDelay
  .filter(col("UniqueCarrier").isin("AS","AQ","HA","F9"))   # Filter UniqueCarrier to only "AS", "AQ", "HA" and "F9"
  .filter(col("TotalDelay") > 0)  # TotalDelay to non-zero values
)

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png">Plot a LIVE graph</h2>

Plot `delaysDF` and give the stream the name "delays_python"

Once the data is loaded, render a line graph with 
* **Keys** is set to `DepartureAt`
* **Series groupings** is set to `UniqueCarrier`
* **Values** is set to `TotalDelay`


In [ ]:
initialDF.isStreaming

Out[26]: True

In [ ]:
# TODO
myStreamName = "delays_python"
display(delaysDF, streamName = myStreamName)

In [ ]:
# TEST - Run this cell to test your solution.
count = 0
for s in spark.streams.active:
  if (s.name == myStreamName):
    count = count + 1

dbTest("SS-02-runningCount", 1, count)

print("Tests passed!")

Tests passed!

When you are done, stop the stream:

In [ ]:
stopAllStreams()

Stopping the stream delays_python.
The stream delays_python was stopped.

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png">Write Stream</h2>

Write the stream to an in-memory table
1. Use appropriate `format`
2. For this exercise, we want to append new records to the results table
3. We have configured a 15 second trigger
4. Name the query "delays_python"
5. Start the query
6. Assign the query to `delayQuery`

In [ ]:
# TODO
delayQuery = (delaysDF.writeStream              # From the DataFrame get the DataStreamWriter
             .format("memory")             # Specify the sink format as "memory"
             .outputMode("append")               # Configure the output mode as "append"
             .queryName(myStreamName) # Name the query with myStreamName
             .trigger(processingTime = "15 seconds")               # Use a 15 second trigger
             .start()               # Start the query
)

In [ ]:
# TEST - Run this cell to test your solution.
dbTest("SS-02-isActive", True, delayQuery.isActive)
dbTest("SS-02-name", myStreamName, delayQuery.name)
# The query's trigger is not available via the Python API

print("Tests passed!")

Tests passed!

Wait until stream is done initializing...

In [ ]:
myStreamName = "delays_python"

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.

*   List item
*   List item

png"> Exercise 5: Stop streaming jobs</h2>

Before we can conclude, we need to shut down all active streams.

In [ ]:
# TODO
for streams in spark.streams.active:# Iterate over all active streams
  try:
    print("stopping " + streams.name)  # A little console output
    streams.stop()     # Stop the stream
    
  except Exception as e:
    print("some error occured")

stopping delays_python

In [ ]:
# TEST - Run this cell to test your solution.
dbTest("SS-02-numActiveStreams", 0, len(spark.streams.active))

print("Tests passed!")

Tests passed!

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [ ]:
%run "../Includes/Classroom-Cleanup"

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Next Steps</h2>

Start the next part, [Time Windows]($../SS 03 - Time Windows).